# Tidy Data Project
The aim of this notebook is to clean and organize data regarding the **federal R&D budget** by applying the **tidy data principles**, followed by the execution of basic exploratory data analyses.

Let's start by downloading and importing the data we are to tidy and study, which is the R&D spending of various government subsidiaries throughout 1976 to 2017 and the GDP of the subsequent year.

In [204]:
# download dataset
# please follow the instructions included in the readme for this notebook under the subtitle "Downloading 
# and moving the dataset"

In [205]:
import pandas as pd

# import dataset into notebook
df_fedrd = pd.read_csv('fed_rd_year&gdp.csv')

### Data Cleaning and Tidy Process

Contrary to the what a layperson may assume, much of a data scientist's time is not spent observing and drawing conclusions based on the collected data, but rather spent on tidying and organizing the numbers. With many datasets being collected in bizarre, confusing manners, this step is diligent commitment to this long process is necessary so that the *data is structured in a standardized way* where each cell has a clear value tied with specific variables and observations. <br>

To form tidy data, we must make sure that **(1) each variable forms a column**, **(2) each observation forms a row**, and finally **(3) each type of observational unit forms a table**. 
<br><br>
Let us take a peek at the first couple of rows of the original dataset, which is named `df_fedrd` here.

In [206]:
print(df_fedrd.head())

  department  1976_gdp1790000000000.0  1977_gdp2028000000000.0  \
0        DHS                      NaN                      NaN   
1        DOC             8.190000e+08             8.370000e+08   
2        DOD             3.569600e+10             3.796700e+10   
3        DOE             1.088200e+10             1.374100e+10   
4        DOT             1.142000e+09             1.095000e+09   

   1978_gdp2278000000000.0  1979_gdp2570000000000.0  1980_gdp2797000000000.0  \
0                      NaN                      NaN                      NaN   
1             8.710000e+08             9.520000e+08             9.450000e+08   
2             3.702200e+10             3.717400e+10             3.700500e+10   
3             1.566300e+10             1.561200e+10             1.522600e+10   
4             1.156000e+09             1.004000e+09             1.048000e+09   

   1981_gdp3138000000000.0  1982_gdp3314000000000.0  1983_gdp3541000000000.0  \
0                      NaN                

To put meaning to each value, we must identify the variable and observation that it belongs to. 

For this dataset, the variables that we must make into individual columns are department, year, R&D spending, and GDP. We can see above that department is in its proper formation, but R&D spending is under a column name that includes both the year and the year's GDP. Using the tidy data principles aforementioned, we will reorganize the values into a structured, readable manner with all appropriate columns and their observations.

In [207]:
## tidy data following the tidy data principles

# melt the dataframe so that 'year' and 'R&D spending' are two separate columns
df_fedrd_melted = pd.melt(df_fedrd,
                          id_vars = ['department'],
                          value_vars = df_fedrd.columns[1:],
                          var_name = 'year',
                          value_name = 'R&D spending')

# split the 'year' column using str.split into a year and GDP column
df_fedrd_melted[['year', 'GDP']] = df_fedrd_melted['year'].str.split(pat = '_', expand = True)

# clean the GDP column to remove unnecessary characters and make it more readable
# for this dataset we are removing the 'gdp' before each value under the GDP column
df_fedrd_melted['GDP'] = df_fedrd_melted['GDP'].str.replace('gdp','')

# print the first couple of columns of the tidied dataset
# for the full dataset, delete the ".head()" in the code line below # warning: output will be truncated
print(df_fedrd_melted.head())

  department  year  R&D spending              GDP
0        DHS  1976           NaN  1790000000000.0
1        DOC  1976  8.190000e+08  1790000000000.0
2        DOD  1976  3.569600e+10  1790000000000.0
3        DOE  1976  1.088200e+10  1790000000000.0
4        DOT  1976  1.142000e+09  1790000000000.0


### Pivot Tables

These tables are simple tools that can be used to dive deeper into specific variables and identify patterns within the tidied data using aggregation functions such as mean, max, etc. 
<br><br>
Here we will create and look at two tables using the `pd.pivot_table` function containing the following:
1. R&D spending used by each department on average from 1967 to 2017
2. average R&D spending used by each department from 1967-2017 as a percentage of the nation's GDP

In [208]:
## create a pivot table that shows 'R&D spending' used by each department on average throughout all years
# drop the GDP column using 'iloc' as it is not needed in this specific pivot table
pivot_table_avg_rdspending = pd.pivot_table(df_fedrd_melted.iloc[:, [0, 1, 2]],
               index = 'department',
               values = 'R&D spending',
               aggfunc = 'mean')

print(f'Average R&D Spending by Department from 1976-2017')
print(pivot_table_avg_rdspending)

## create a pivot table that shows each department's R&D spending as a percentage of the GDP from greatest to least

# drop columns that have a NaN value for R&D spending to prevent following calculation TypeError
df_fedrd_melted.dropna(subset = ['R&D spending', 'GDP'], inplace = True)

# make the values of both R&D spending and GDP columns numeric to prevent TypeError
df_fedrd_melted['R&D spending'] = pd.to_numeric(df_fedrd_melted['R&D spending'], errors='coerce')
df_fedrd_melted['GDP'] = pd.to_numeric(df_fedrd_melted['GDP'], errors='coerce')

# calculate the R&D as a percentage of GDP
df_fedrd_melted['R&D as a percent of GDP'] = (df_fedrd_melted['R&D spending'] / df_fedrd_melted['GDP']) * 100

pivot_table_rdspending_pc_GDP = pd.pivot_table(df_fedrd_melted,
                                               index = 'department',
                                               values = 'R&D as a percent of GDP',
                                               aggfunc = 'mean')

# sort the results from greatest percentage to smallest percentage
pivot_table_rdspending_pc_GDP_sorted = pivot_table_rdspending_pc_GDP.sort_values(by = 'R&D as a percent of GDP', ascending=False)

print() # print line break between two pivot tables
print(f'R&D Spending by Department as a % of GDP from 1976-2017 (Greatest to Least)')
print(pivot_table_rdspending_pc_GDP_sorted)

Average R&D Spending by Department from 1976-2017
            R&D spending
department              
DHS         9.948750e+08
DOC         1.231500e+09
DOD         6.468519e+10
DOE         1.188338e+10
DOT         9.177857e+08
EPA         7.504286e+08
HHS         2.229676e+10
Interior    9.005714e+08
NASA        1.214026e+10
NIH         2.111757e+10
NSF         4.038810e+09
Other       1.494762e+09
USDA        2.380810e+09
VA          7.338810e+08

R&D Spending by Department as a % of GDP from 1976-2017 (Greatest to Least)
            R&D as a percent of GDP
department                         
DOD                        0.931092
HHS                        0.269553
NIH                        0.251655
DOE                        0.207941
NASA                       0.198547
NSF                        0.054083
USDA                       0.036455
Other                      0.025885
DOC                        0.017452
Interior                   0.016046
DOT                        0.015749
EPA  